In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd 
import numpy as np 
from datetime import datetime

In [3]:
dataset = pd.read_csv('./Dataset/Dataset.csv')

In [4]:
dataset.shape

(3596299, 25)

In [5]:
dataset.head()

,SrcAddr,DstAddr,Sport,Dport,sTos,dTos,Dir,State,Dur,TotPkts,...,StartTimeMinute,StartTimeSecond,Proto_icmp,Proto_rtcp,Proto_rtp,Proto_tcp,Proto_udp,Proto_udt,isBotnet,isSpam
0,86658,47,6907,300,0,0,0,250,0.000000,1,...,0,0,1,0,0,0,0,0,1,0
1,587582,6,667,10,0,0,3,9,0.132290,2,...,0,0,0,0,0,0,1,0,0,0
2,587582,19,48,156707,0,0,5,124,3570.602295,260,...,0,0,0,0,0,1,0,0,0,0
3,7,69,4169,2,0,0,5,25,3467.739746,345,...,0,0,0,0,0,1,0,0,0,0
4,9327,1996,102,625148,0,0,0,78,115.104431,10,...,0,0,0,0,0,1,0,0,0,0


In [6]:
X_botnet = dataset.drop(columns=['isBotnet', 'isSpam', 'Label', 'BotnetName'])
y_botnet = dataset['isBotnet']

In [ ]:
spam_dataset = dataset[~dataset['Label'] == 0]
X_spam = spam_dataset.drop(columns=['isBotnet', 'isSpam', 'Label'])
y_spam = spam_dataset['isSpam']

# BPSO

In [7]:
# BPSO

# Sigmoid function to convert velocity into probabilities
def sigmoid(v):
    return 1 / (1 + np.exp(-v))

# Function to update position (binary)
def update_position(velocity):
    probability = sigmoid(velocity)
    new_position = np.random.rand(len(velocity)) < probability
    return new_position.astype(int)  # Convert boolean to binary (0 or 1)

# Fitness function (based on model accuracy)
def fitness_function(selected_features, X, y):
    # Ensure selected_features is a valid boolean mask
    selected_features = selected_features.astype(bool)
    
    if np.sum(selected_features) == 0:  # Avoid selecting no features
        return 1.0  # Poor fitness if no features are selected
    
    # Select features based on the boolean mask
    # If X is a DataFrame, you can use X.loc[:, selected_features]
    # If X is a NumPy array, simply use X[:, selected_features]
    if isinstance(X, np.ndarray):
        selected_X = X[:, selected_features]
    elif isinstance(X, pd.DataFrame):
        selected_X = X.loc[:, selected_features]

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(selected_X, y, test_size=0.25, random_state=42)

    # Train a decision tree classifier
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)

    # Predict and calculate accuracy
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return 1 - accuracy  # Return 1 - accuracy to minimize the fitness

# BPSO Algorithm for Feature Selection
def bpso_feature_selection(X, y, n_particles=50, max_iter=100, w=0.5, c1=1.5, c2=1.5):
    n_features = X.shape[1]
    
    # Initialize particle positions (binary) and velocities
    positions = np.random.randint(2, size=(n_particles, n_features))
    velocities = np.random.uniform(-1, 1, (n_particles, n_features))
    
    # Initialize personal best positions and global best position
    pbest_positions = positions.copy()
    pbest_fitness = np.array([fitness_function(positions[i], X, y) for i in range(n_particles)])
    
    gbest_position = pbest_positions[np.argmin(pbest_fitness)]
    gbest_fitness = np.min(pbest_fitness)
    
    # BPSO main loop
    for iteration in range(max_iter):
        for i in range(n_particles):
            # Update velocity
            r1, r2 = np.random.rand(n_features), np.random.rand(n_features)
            velocities[i] = (w * velocities[i] +
                             c1 * r1 * (pbest_positions[i] - positions[i]) +
                             c2 * r2 * (gbest_position - positions[i]))
            
            # Update position (convert velocity to binary)
            positions[i] = update_position(velocities[i])
            
            # Calculate new fitness
            fitness = fitness_function(positions[i], X, y)
            
            # Update personal best if the current position is better
            if fitness < pbest_fitness[i]:
                pbest_positions[i] = positions[i]
                pbest_fitness[i] = fitness
        
        # Update global best if a better solution is found
        best_particle_idx = np.argmin(pbest_fitness)
        if pbest_fitness[best_particle_idx] < gbest_fitness:
            gbest_position = pbest_positions[best_particle_idx]
            gbest_fitness = pbest_fitness[best_particle_idx]
        
        print(f"Iteration {iteration+1}/{max_iter}, Best Fitness: {gbest_fitness}")
    
    return gbest_position, 1 - gbest_fitness  # Return best feature subset and corresponding accuracy

In [8]:
# DT-1
best_features, best_accuracy = bpso_feature_selection(X_botnet, y_botnet)

Iteration 1/100, Best Fitness: 0.0
Iteration 2/100, Best Fitness: 0.0
Iteration 3/100, Best Fitness: 0.0
Iteration 4/100, Best Fitness: 0.0
Iteration 5/100, Best Fitness: 0.0
Iteration 6/100, Best Fitness: 0.0
Iteration 7/100, Best Fitness: 0.0
Iteration 8/100, Best Fitness: 0.0
Iteration 9/100, Best Fitness: 0.0
Iteration 10/100, Best Fitness: 0.0
Iteration 11/100, Best Fitness: 0.0
Iteration 12/100, Best Fitness: 0.0
Iteration 13/100, Best Fitness: 0.0
Iteration 14/100, Best Fitness: 0.0
Iteration 15/100, Best Fitness: 0.0
Iteration 16/100, Best Fitness: 0.0
Iteration 17/100, Best Fitness: 0.0
Iteration 18/100, Best Fitness: 0.0
Iteration 19/100, Best Fitness: 0.0
Iteration 20/100, Best Fitness: 0.0
Iteration 21/100, Best Fitness: 0.0
Iteration 22/100, Best Fitness: 0.0
Iteration 23/100, Best Fitness: 0.0
Iteration 24/100, Best Fitness: 0.0
Iteration 25/100, Best Fitness: 0.0
Iteration 26/100, Best Fitness: 0.0
Iteration 27/100, Best Fitness: 0.0
Iteration 28/100, Best Fitness: 0.0
I

In [9]:
best_features, best_accuracy

(array([1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1]), 1.0)

In [ ]:
# DT-2
best_features_spam, best_accuracy_spam = bpso_feature_selection(X_spam, y_spam)